# Webcrawler um Analystenempfehlungen zur Siemens AG zu extrahieren

> **Disclaimer:**  
> Bitte beachten Sie, dass **HTML-Klassen** regelmäßig angepasst werden müssen, da sich diese im HTML-Code der Website ändern können.  
> Die anzupassenden Stellen wurden im Code kommentiert.

In [2]:
# Importieren der benötigten Bibliotheken
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from parsel import Selector
import openpyxl
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

## 1. Webdriver initialisieren

In [4]:
# Die URL zu finanznachrichten.de
url = 'https://www.finanznachrichten.de/nachrichten-aktien/siemens-ag.htm'

# Starte den WebDriver-Server auf einem bestimmten Port (z.B. 9517)
driver = webdriver.Chrome()

# Auf die Seite mit der Zustimmungserklärung gehen
driver.get(url)

# Warte bis das iframe geladen ist (du kannst diesen Schritt überspringen, wenn es nicht notwendig ist)
WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.ID, "sp_message_iframe_888903")))

# Jetzt kannst du Elemente innerhalb des iframes finden und darauf zugreifen
element_inside_iframe = driver.find_element(By.XPATH, '//*[@id="notice"]/div[3]/div[2]/button')

# Beispiel: Klicke auf ein Element innerhalb des iframes
element_inside_iframe.click()

# Wechsle zurück zum Hauptinhalt (falls notwendig)
driver.switch_to.default_content()

In [5]:
# Klick auf Button "Empfehlungen"
recommendations = driver.find_element(By.XPATH, '//*[@id="W115"]/ul/li[3]')
recommendations.click()

In [1]:
# Nicht mehr notwendig
# table = driver.find_element(By.CLASS_NAME, 'table-hoverable.table-alternating-rows')

## 2. Crawling 1 durchführen und Ergebnisse in Liste "data" speichern

> **Crawling 1:**  
> Crawling von "date", "title" und "link" (Llink zur Analystenbewertung)

In [19]:
data = []
max_iterations = 8
iteration_count = 0

while iteration_count < max_iterations:
    # Iteriere über die Zeilen der Tabelle
    rows = driver.find_elements(By.CSS_SELECTOR, "tr")

    for row in rows:
        if row.text:  # Überprüfen, ob die Zeile Text enthält
            # Für jede Zeile, hole die Zellen
            dates = row.find_elements(By.CLASS_NAME, 'zentriert.ft-vsbl.ft-first-column')
            titles = row.find_elements(By.CLASS_NAME, 'zl.ft-vsbl')
            link_elements = row.find_elements(By.XPATH, ".//a[@href]")
            for i in range(len(dates)):
                    date = dates[i].text
                    title = titles[i].text
                    link = link_elements[i].get_attribute("href")
                    data.append([date, title, link])

    # Überprüfe, ob der "Weiter"-Button vorhanden ist
    next_button = driver.find_elements(By.CLASS_NAME, "fnxui.href.cursor_pointer")[-1] # Immer letztes Element der Seitenzahlen auswählen "Weiter-Button"
    
    if next_button.is_enabled():
        next_button.click()
        time.sleep(25)
        iteration_count += 1
    else:
        break  # Beende die Schleife, wenn kein "Weiter"-Button mehr vorhanden ist

# Jetzt können Sie mit der Datenliste weiterarbeiten
print(data)

[['Fr', "DEUTSCHE BANK RESEARCH stuft SIEMENS AG auf 'Buy'", 'https://www.finanznachrichten.de/nachrichten-2024-02/61304452-deutsche-bank-research-stuft-siemens-ag-auf-buy-322.htm'], ['23.01.', "BERNSTEIN RESEARCH stuft SIEMENS AG auf 'Outperform'", 'https://www.finanznachrichten.de/nachrichten-2024-01/61204655-bernstein-research-stuft-siemens-ag-auf-outperform-322.htm'], ['22.01.', "JPMORGAN stuft SIEMENS AG auf 'Overweight'", 'https://www.finanznachrichten.de/nachrichten-2024-01/61196529-jpmorgan-stuft-siemens-ag-auf-overweight-322.htm'], ['22.01.', "DZ BANK stuft SIEMENS AG auf 'Kaufen'", 'https://www.finanznachrichten.de/nachrichten-2024-01/61195544-dz-bank-stuft-siemens-ag-auf-kaufen-322.htm'], ['15.01.', "JPMORGAN stuft SIEMENS AG auf 'Overweight'", 'https://www.finanznachrichten.de/nachrichten-2024-01/61137829-jpmorgan-stuft-siemens-ag-auf-overweight-322.htm'], ['15.01.', "RBC stuft SIEMENS AG auf 'Outperform'", 'https://www.finanznachrichten.de/nachrichten-2024-01/61137498-rbc-

## 3. Excel-Liste vorbereiten und (Zwischen-)Speichern der Ergebnisse

In [ ]:
# Excel-Datei mit Openpyxl vorbereiten, um die Reviews hier später speichern zu können
wb= openpyxl.load_workbook("analyst_recommendations.xlsx")
sheetname=wb.get_sheet_names()[0]
sheet=wb[sheetname]
sheet.title ="recommendations"
sheet.append(["Date", "Title", "Link", "Recommendation"])
wb.save("analyst_recommendations.xlsx")

In [20]:
# Reviews aus der Liste in die Excel-Datei speichern
for item in data:
        sheet.append(item)

# Excel-Datei speichern und schließen
wb.save("analyst_recommendations.xlsx")
wb.close()

## 4. Crawling 2 durchführen und Ergebnisse in DataFrame speichern

> **Crawling 2:**  
> Crawling der Analystenbewertungen auf Basis der vorher ermittelten Links

In [25]:
# df mit "date", "title", "link" erstellen
df = pd.DataFrame(data, columns=["date", "title", "link"])
df

,date,title,link
0,Fr,DEUTSCHE BANK RESEARCH stuft SIEMENS AG auf 'Buy',https://www.finanznachrichten.de/nachrichten-2...
1,23.01.,BERNSTEIN RESEARCH stuft SIEMENS AG auf 'Outpe...,https://www.finanznachrichten.de/nachrichten-2...
2,22.01.,JPMORGAN stuft SIEMENS AG auf 'Overweight',https://www.finanznachrichten.de/nachrichten-2...
3,22.01.,DZ BANK stuft SIEMENS AG auf 'Kaufen',https://www.finanznachrichten.de/nachrichten-2...
4,15.01.,JPMORGAN stuft SIEMENS AG auf 'Overweight',https://www.finanznachrichten.de/nachrichten-2...
...,...,...,...
142,09.02.23,DEUTSCHE BANK RESEARCH stuft SIEMENS AG auf 'Buy',https://www.finanznachrichten.de/nachrichten-2...
143,09.02.23,BARCLAYS stuft SIEMENS AG auf 'Underweight',https://www.finanznachrichten.de/nachrichten-2...
144,09.02.23,GOLDMAN SACHS stuft SIEMENS AG auf 'Buy',https://www.finanznachrichten.de/nachrichten-2...
145,09.02.23,UBS stuft SIEMENS AG auf 'Buy',https://www.finanznachrichten.de/nachrichten-2...


In [26]:
# Definition einer Funktion, um den text (Analystenbewertung) auf Basis des Links zu extrahieren
def get_recommendation(website_url):
    website_url = website_url

    # Verwenden Sie einen WebDriver (z.B., ChromeDriver) für die Ausführung von JavaScript auf der Seite
    driver_1 = webdriver.Chrome()
    driver_1.get(website_url)

    text_element = driver_1.find_element(By.ID, "artikelTextPuffer")
    text = text_element.text.split('\n', 1)[0]

    # Schließen Sie den WebDriver, um Ressourcen freizugeben
    driver_1.quit()

    return text

In [31]:
# Schleife, um alle Links durchzugehen und Analystenbewertungen zu extrahieren
texts = []

for link in df["link"]:
    recommendation_text = get_recommendation(link)
    
    texts.append(recommendation_text)
    print(recommendation_text)

    time.sleep(30)

FRANKFURT (dpa-AFX Analyser) - Deutsche Bank Research hat die Einstufung für Siemens auf "Buy" mit einem Kursziel von 195 Euro belassen. Siemens Healthineers habe bei der Zahlenvorlage mit der Auftragsentwicklung überzeugt, schrieb Analyst Gael de-Bray in einer am Freitag vorliegenden Studie./mis/zb
NEW YORK (dpa-AFX Analyser) - Das US-Analysehaus Bernstein Research hat die Einstufung für Siemens vor den Quartalsberichten aus dem Investitionsgütersektor auf "Outperform" mit einem Kursziel von 190 Euro belassen. Die Unternehmen dürften die Erwartungen wohl erfüllen oder auch übertreffen, schrieb Analyst Nicholas Green in einer am Dienstag vorliegenden Sektorstudie. Mit Siemens setzten Anleger auf Qualität und auf die großen Trends Automation, Elektrifizierung und Industrie./bek/edh
NEW YORK (dpa-AFX Analyser) - Die US-Bank JPMorgan hat die Einstufung für Siemens vor Zahlen zum ersten Geschäftsquartal auf "Overweight" mit einem Kursziel von 195 Euro belassen. Seine Umsatz- und Ergebnispr

In [32]:
# Analstenbewertungen als Spalte in df aufnehmen
df["recommendation"] = texts
df

,date,title,link,recommendation
0,Fr,DEUTSCHE BANK RESEARCH stuft SIEMENS AG auf 'Buy',https://www.finanznachrichten.de/nachrichten-2...,FRANKFURT (dpa-AFX Analyser) - Deutsche Bank R...
1,23.01.,BERNSTEIN RESEARCH stuft SIEMENS AG auf 'Outpe...,https://www.finanznachrichten.de/nachrichten-2...,NEW YORK (dpa-AFX Analyser) - Das US-Analyseha...
2,22.01.,JPMORGAN stuft SIEMENS AG auf 'Overweight',https://www.finanznachrichten.de/nachrichten-2...,NEW YORK (dpa-AFX Analyser) - Die US-Bank JPMo...
3,22.01.,DZ BANK stuft SIEMENS AG auf 'Kaufen',https://www.finanznachrichten.de/nachrichten-2...,FRANKFURT (dpa-AFX Analyser) - Die DZ Bank hat...
4,15.01.,JPMORGAN stuft SIEMENS AG auf 'Overweight',https://www.finanznachrichten.de/nachrichten-2...,NEW YORK (dpa-AFX Analyser) - Die US-Bank JPMo...
...,...,...,...,...
142,09.02.23,DEUTSCHE BANK RESEARCH stuft SIEMENS AG auf 'Buy',https://www.finanznachrichten.de/nachrichten-2...,FRANKFURT (dpa-AFX Analyser) - Deutsche Bank R...
143,09.02.23,BARCLAYS stuft SIEMENS AG auf 'Underweight',https://www.finanznachrichten.de/nachrichten-2...,LONDON (dpa-AFX Analyser) - Die britische Inve...
144,09.02.23,GOLDMAN SACHS stuft SIEMENS AG auf 'Buy',https://www.finanznachrichten.de/nachrichten-2...,NEW YORK (dpa-AFX Analyser) - Die US-Investmen...
145,09.02.23,UBS stuft SIEMENS AG auf 'Buy',https://www.finanznachrichten.de/nachrichten-2...,ZÜRICH (dpa-AFX Analyser) - Die Schweizer Groß...


In [34]:
# Speichern des finalen df in Excel
df.to_excel("data_analyst_recommendations.xlsx", index=False)